# LangChain Agent

From 
- https://python.langchain.com/v0.1/docs/modules/agents/quick_start/
- https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/

In [1]:
import os

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.agents import tool
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent, create_openai_tools_agent
from langchain import hub

In [2]:
load_dotenv()

True

## Set up LLM

In [3]:
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0
)

## Define tools

In [4]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def get_temperature(location: str) -> int:
    """Returns the live temperature at a given location in degrees Celsius."""
    return 25

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2

In [5]:
# get_word_length.invoke("abc")

In [6]:
tools = [magic_function, get_word_length]

## Agent instruction

In [7]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Create agent

In [8]:
# Create agent
agent = create_tool_calling_agent(llm, tools, prompt)
# agent = create_openai_tools_agent(llm, tools, prompt)

In [9]:
# Create executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Invoke

In [10]:
# agent_executor.invoke({"input": "What is the current temperature in Cape Town? Use your `get_temperature('Cape Town')` to get the answer."})
# agent_executor.invoke({"input": "what is the value of magic_function(3)?"})
agent_executor.invoke({"input": "what is the length of the word 'beach'?"})



> Entering new AgentExecutor chain...
Could not parse tool input: {'arguments': "const get_word_length: functions.get_word_length = ({ word }) => {\n  return word.length;\n};\n\nget_word_length({ word: 'beach' });", 'name': 'functions'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'arguments': "from typing import Dict, Any\n\ndef get_word_length(args: Dict[str, Any]) -> int:\n    word = args['word']\n    return len(word)\n\nget_word_length({'word': 'beach'})", 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'arguments': "from typing import Dict, Any\n\ndef get_word_length(args: Dict[str, Any]) -> int:\n    word = args['word']\n    return len(word)\n\nget_word_length({'word': 'beach'})", 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete responseCould not parse tool input: {'arguments': "from typing import Dict, Any\n\ndef get_wo

{'input': "what is the length of the word 'beach'?",
 'output': 'Agent stopped due to max iterations.'}